In [13]:
import json
import glob
import os
import datetime
import re
import numpy as np
import pandas as pd
from pathlib import Path
from slack_data_loader2 import SlackLoader2, normalize_links
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem.snowball import EnglishStemmer, RussianStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

import eli5
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge, RidgeClassifier
from sklearn import linear_model
from sklearn.pipeline import FeatureUnion
from sklearn.metrics import mean_squared_log_error, roc_auc_score
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression, RidgeClassifierCV
from sklearn.grid_search import GridSearchCV

from sklearn.model_selection import StratifiedShuffleSplit, cross_val_score

from sklearn.linear_model import RidgeClassifierCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor

from operator import itemgetter

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### data
* "channel"
* "start_message_text"
* "thread_text" (__eou__ __eot__)
* "message_type" = ["top", "first_thread", "thread"]
* "attachment" = 1|0
* "message_number" = (1 - top,)
* "message_text" 
### _target_
emoji, count

In [61]:
BASE_DIR = Path("/home/algis/repos/personal/MOOC/ODS_dump/input/export_Feb_8_2018")

channels = ['_jobs', 'career']

loader = SlackLoader2(BASE_DIR, exclude_channels=[])#, only_channels=channels)
print(len(loader.messages))
print(len(loader.threads))
df = pd.DataFrame(loader.threads).set_index(["channel_name", "start_ts"])

  2%|▏         | 13080/751861 [00:01<01:33, 7877.32it/s] | 3477/751861 [00:00<00:21, 34749.40it/s]

Empty text {'subtype': 'bot_message', 'bot_id': 'B09DR8T8S', 'attachments': [{'pretext': 'There are no events today.', 'markdwn_in': ['pretext']}], 'channel': 'C04422A5C', 'type': 'message', 'ts': 1440172799.000431, 'text': None, 'dt': datetime.datetime(2015, 8, 21, 18, 59, 59, 431), 'channel_name': '_meetings'}
Empty text {'subtype': 'bot_message', 'bot_id': 'B09DR8T8S', 'attachments': [{'pretext': 'There are no events today.', 'markdwn_in': ['pretext']}], 'channel': 'C04422A5C', 'type': 'message', 'ts': 1440993599.000191, 'text': None, 'dt': datetime.datetime(2015, 8, 31, 6, 59, 59, 191), 'channel_name': '_meetings'}


 54%|█████▍    | 409268/751861 [21:25<36:59, 154.35it/s]              | 14638/751861 [00:01<01:50, 6675.49it/s] 46%|████▋     | 348644/751861 [14:42<17:01, 394.90it/s]

Empty text {'subtype': 'bot_message', 'bot_id': 'B4WF6UYSW', 'attachments': [{'color': '6ecadc', 'mrkdwn_in': ['fields'], 'title': 'Во сколько устраивать завтраки?', 'fields': [{'short': False, 'value': ':one: 8.30\n\n\n', 'title': ''}, {'short': False, 'value': ':two: 9.00    `5`\negor_labintcev, itjune, anotherbugmaster, chipolinka, kolju\n\n', 'title': ''}, {'short': False, 'value': ':three: 9.30\n\n\n', 'title': ''}, {'short': False, 'value': ':four: 10.00    `2`\nntarasov, chslv\n\n', 'title': ''}, {'short': False, 'value': ':five: 10.30\n\n\n', 'title': ''}], 'id': 1, 'fallback': 'Open Slack to cast your vote in this Simple Poll'}, {'actions': [{'style': '', 'value': '1', 'text': ':one:', 'type': 'button', 'name': 'vote', 'id': '1'}, {'style': '', 'value': '2', 'text': ':two:', 'type': 'button', 'name': 'vote', 'id': '2'}, {'style': '', 'value': '3', 'text': ':three:', 'type': 'button', 'name': 'vote', 'id': '3'}, {'style': '', 'value': '4', 'text': ':four:', 'type': 'button', 'n

 55%|█████▍    | 411162/751861 [21:38<33:27, 169.72it/s] 54%|█████▍    | 409307/751861 [21:25<33:13, 171.86it/s]

...

Empty text {'reactions_': None, 'thread_ts': '1512733041.000120', 'channel': 'C04PE5M8V', 'ts': 1512733041.00012, 'type': 'message', 'dt': datetime.datetime(2017, 12, 8, 14, 37, 21, 120), 'attachments': [{'id': 1, 'video_html_height': 225, 'author_name': 'Alister', 'service_url': 'https://www.youtube.com/', 'thumb_height': 360, 'thumb_width': 480, 'author_link': 'https://www.youtube.com/user/IRSOG1', 'from_url': 'https://opendatascience.slack.com/archives/C0SGCGB52/p1512732901000319?thread_ts=1512656271000044&cid=C0SGCGB52', 'service_icon': 'https://a.slack-edge.com/2089/img/unfurl_icons/youtube.png', 'color': 'D0D0D0', 'title': "Ali Rahimi's talk at NIPS(NIPS 2017 Test-of-time award presentation)", 'is_share': True, 'video_html': '<iframe width="400" height="225" src="https://www.youtube.com/embed/Qi1Yry33TQE?feature=oembed&autoplay=1&iv_load_policy=3" frameborder="0" gesture="media" allow="encrypted-media" allowfullscreen></iframe>', 'title_link': 'https://m.youtube.com/watch?v=Qi1Yr

 98%|█████████▊| 736258/751861 [1:27:04<04:13, 61.48it/s]  3%|█████████▎| 699834/751861 [1:16:40<11:53, 72.87it/s]

Empty text {'subtype': 'bot_message', 'bot_id': 'B8KECPCLC', 'attachments': [{'color': '6ecadc', 'mrkdwn_in': ['fields'], 'title': 'In 6 months bitcoin will be', 'fields': [{'short': False, 'value': ':one: higher    `12`\n<@U8G3S156E>, <@U50EVF538>, <@U1Q0B8KMJ>, <@U4SKDV2HF>, <@U4GCG7FGR>, <@U7BSFBQ1M>, <@U28J6CP29>, <@U298B1T19>, <@U0PEXP9PH>, <@U489DDYLU>, <@U5MN8V5T3>, <@U8313GG7M>\n\n', 'title': ''}, {'short': False, 'value': ':two: lower    `12`\n<@U74GVQMTM>, <@U4U5DNNPJ>, <@U04ELQZAU>, <@U49NJJXNJ>, <@U1LGHMGPM>, <@U36KSMGDN>, <@U5F8LG1H8>, <@U6QD48HHS>, <@U5GRLNVKJ>, <@U6BS23P47>, <@U7D8T5MHP>, <@U8TCF6RCG>\n\n', 'title': ''}, {'short': False, 'value': ':three: same as now    `4`\n<@U8WEYGQAH>, <@U19VD9AQH>, <@U6XAU8ZNW>, <@U4WPS5X8X>\n\n', 'title': ''}], 'id': 1, 'fallback': 'Open Slack to cast your vote in this Simple Poll'}, {'actions': [{'style': '', 'value': '1', 'text': ':one:', 'type': 'button', 'name': 'vote', 'id': '1'}, {'style': '', 'value': '2', 'text': ':two:', 't

100%|██████████| 751861/751861 [1:31:48<00:00, 136.49it/s]98%|█████████▊| 736274/751861 [1:27:05<04:48, 54.11it/s]


751861
40216


In [3]:
%%time

#df = df.set_index(["channel_name", "start_ts"])
#df[df["msg_counter"] > 1].to_csv("threads.csv")
file_cache = "all_threads.pkl.compress"
#df.to_pickle(file_cache, compression="gzip")
#rt = pd.read_pickle("s1.pkl.bz2")

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.96 µs


In [4]:
%%time

df_readed = pd.read_pickle(file_cache, compression="gzip")

CPU times: user 624 ms, sys: 52 ms, total: 676 ms
Wall time: 686 ms


In [5]:
%%time

def get_ktop_reactions(df, k=20):
    df["reactions_"] = df["reactions_"].apply(lambda x : dict(x) )
    df2 = df["reactions_"].apply(pd.Series)
    df3 = pd.concat([df, df2], axis=1).drop('reactions_', axis=1)
    react_stat = df3.drop(["msg_counter"], axis=1).describe().transpose().sort_values(["count"], ascending=False)
    #df_[df_["count"] >= 100] #get by threshold
    cols_to_del = react_stat[k:].index.values
    ktop_cols = react_stat[:k].index.values
    was_rows = df3.shape[0]
    df3 = df3.drop(cols_to_del, axis=1).dropna(axis=0, subset=ktop_cols, how='all')
    print("{} rows droped.".format(was_rows-df3.shape[0]))
    return df3, ktop_cols

reacts, ktop_cols = get_ktop_reactions(df_readed)

13408 rows droped.
CPU times: user 13.3 s, sys: 524 ms, total: 13.9 s
Wall time: 13.9 s


In [6]:
reacts

end_ts  \
channel_name        start_ts                                                
_general            2015-07-23 19:10:16.000091                        NaT   
                    2015-07-23 19:22:55.000097                        NaT   
                    2015-08-26 14:06:00.000012                        NaT   
_random_flood       2015-09-09 14:29:57.000056                        NaT   
                    2015-09-16 09:13:30.000283                        NaT   
                    2015-09-16 10:36:45.000314                        NaT   
_meetings           2015-09-18 13:56:34.000333                        NaT   
                    2015-09-18 20:21:33.000397                        NaT   
                    2015-09-19 01:00:43.000400                        NaT   
                    2015-09-21 17:32:12.000442                        NaT   
_random_flood       2015-09-22 14:19:34.000542                        NaT   
                    2015-09-23 15:08:59.000672                        NaT   
                    2015-09-25 01:21:11.000741                        NaT   
_meetings           2015-09-25 19:52:08.000505                        NaT   
interesting_links   2015-09-27 22:40:38.000152                        NaT   
_random_flood       2015-09-29 14:29:15.000927                        NaT   
_meetings           2015-09-29 15:31:52.000629                        NaT   
interesting_links   2015-09-29 20:02:32.000345                        NaT   
_meetings           2015-09-30 13:57:42.000683                        NaT   
_random_flood       2015-09-30 21:36:51.001016                        NaT   
                    2015-09-30 22:58:10.001065                        NaT   
_meetings           2015-10-01 17:56:09.000725                        NaT   
_random_flood       2015-10-01 21:10:09.001123                        NaT   
_meetings           2015-10-02 13:05:22.000735                        NaT   
_random_flood       2015-10-02 14:05:59.001193                        NaT   
interesting_links   2015-10-03 10:57:19.000372                        NaT   
_random_flood       2015-10-04 20:34:19.000055                        NaT   
                    2015-10-05 13:26:07.000118                        NaT   
                    2015-10-05 14:11:29.000126                        NaT   
_general            2015-10-06 15:45:33.000018                        NaT   
...                                                                   ...   
proj_kaggle_camera  2018-02-08 12:53:34.000025                        NaT   
                    2018-02-08 13:00:43.000167                        NaT   
mlcourse_open       2018-02-08 13:03:45.000045 2018-02-08 13:18:08.000489   
conference          2018-02-08 13:10:23.000198                        NaT   
nlp                 2018-02-08 13:42:33.000099 2018-02-08 13:56:24.000366   
_random_flood       2018-02-08 14:14:06.000234 2018-02-08 14:34:33.000237   
                    2018-02-08 14:14:06.000234 2018-02-08 14:41:12.000391   
proj_kaggle_camera  2018-02-08 14:31:20.000283                        NaT   
_random_flood       2018-02-08 14:44:16.000145 2018-02-08 14:48:06.000218   
career              2018-02-08 15:14:49.000257                        NaT   
                    2018-02-08 15:14:49.000257 2018-02-08 15:16:35.000015   
                    2018-02-08 15:14:49.000257 2018-02-08 15:19:59.000146   
                    2018-02-08 15:14:49.000257 2018-02-08 15:20:15.000314   
                    2018-02-08 15:14:49.000257 2018-02-08 15:29:52.000270   
                    2018-02-08 15:14:49.000257 2018-02-08 15:30:06.000352   
interesting_links   2018-02-08 15:19:17.000168                        NaT   
                    2018-02-08 15:19:17.000168 2018-02-08 16:15:16.000426   
theory_and_practice 2018-02-08 15:25:08.000161 2018-02-08 15:34:05.000008   
proj_kaggle_camera  2018-02-08 15:29:53.000149                        NaT   
career              2018-02-08 15:35:14.000121 2018-02-08 15:40

In [7]:
X = reacts['text'].fillna("_na_").values
y = reacts[ktop_cols].fillna(0).values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)

In [8]:
%%time

ru_stemmer = RussianStemmer()
en_stemmer = EnglishStemmer()
analyzer = CountVectorizer().build_analyzer()

def is_russian_word(w):
    #if not isinstance(w, unicode):
    #    w = w.decode('utf8')
    for c in w:
        if ord(c) >= ord(u'а') and ord(c) <= ord(u'я'):
            return True
    return False

def stemmed_words(doc):
    for w in analyzer(doc):
        yield ru_stemmer.stem(w) if is_russian_word(w) else en_stemmer.stem(w)
        

vectorizer = TfidfVectorizer(min_df=5, analyzer=stemmed_words).fit(X_train)        

CPU times: user 3min 47s, sys: 276 ms, total: 3min 48s
Wall time: 3min 48s


### Other vectorizer

In [26]:
%%time

# we need a custom pre-processor to extract correct field,
# but want to also use default scikit-learn preprocessing (e.g. lowercasing)
default_preprocessor = CountVectorizer().build_preprocessor()

import re, string
re_tok = re.compile('([{}“”¨«»®´·º½¾¿¡§£₤‘’])'.format(string.punctuation))
def tokenize(s): return re_tok.sub(r' \1 ', s).split()
  
vectorizer = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1).fit(X_train)


CPU times: user 25.2 s, sys: 172 ms, total: 25.4 s
Wall time: 25.4 s


In [31]:
%%time
vectorizer = TfidfVectorizer(ngram_range=(1,5), min_df=5, analyzer='char').fit(X_train)      

CPU times: user 1min 46s, sys: 792 ms, total: 1min 47s
Wall time: 1min 47s


In [32]:
for _word, _count in sorted(vectorizer.vocabulary_.items(),key=itemgetter(1),reverse=True)[:20]:
    print(_word,end=' ')
    print(_count)

？анал 641612
？ана 641611
？ан 641610
？а 641609
？ __e 641608
？ __ 641607
？ _ 641606
？  641605
？ 641604
️ __e 641603
️ __ 641602
️ _ 641601
️  641600
️ 641599
ﬁcati 641598
ﬁcat 641597
ﬁca 641596
ﬁc 641595
ﬁ 641594
ツ)_/¯ 641593


In [ ]:
%%time

def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    param_grid = {'C': Cs, 'dual': [False], 'penalty':['l1','l2'], 'class_weight': [None, 'balanced']}
    #param_grid = {'C': Cs}
    model = LinearSVC(random_state=0)
    grid_search = GridSearchCV(model, param_grid, cv=nfolds, n_jobs=1, scoring='roc_auc', verbose=1)
    grid_search.fit(X, y)
    #grid_search.best_params_
    return grid_search
  

gs_result = svc_param_selection(X_train_vec, y_train, 3)

print(gs_result.best_params_, gs_result.best_score_)

In [33]:
%%time

X_train_vec = vectorizer.transform(X_train)


X_val_vec = vectorizer.transform(X_val)

CPU times: user 2min 10s, sys: 416 ms, total: 2min 10s
Wall time: 2min 11s


In [34]:
def pr(y_i, y, x):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)
  
def get_mdl(X, y):
    #y = y.values
    r = np.log(pr(1,y, X) / pr(0,y, X))
    m = LogisticRegression(C=4, dual=True)
    x_nb = X.multiply(r)
    return m.fit(x_nb, y), r  

def get_rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(np.expm1(y_true), np.expm1(y_pred)))

def get_roc_auc(y_true, y_pred):
    return roc_auc_score(y_true, y_pred)
  
def ridge():
    
    return RidgeClassifier(
        solver='auto',
        fit_intercept=True,
        alpha=0.5,
        max_iter=100,
        normalize=False,
        tol=0.05)




def linear_svm():
    C = 0.01
    return LinearSVC(penalty='l1', dual=False, C=C, random_state=0)

def get_y(y_train, index = 1):
    y_ = y_train[:,0]#.reshape(-1,1)
    return y_ #np.where(y_ > 0, 1, 0)

#X_train_vec = X_train_vec#.toarray()
y_ = get_y(y_train)

cv = KFold(n_splits=10, shuffle=False, random_state=42)
models = []
print("start...")
for train_ids, valid_ids in cv.split(X_train_vec):
    #model = linear_svm()
    #model = linear_model.BayesianRidge()
    model = linear_model.Ridge(alpha = .5)
    #model = linear_model.Lasso(alpha = 0.1)
    #model = linear_model.SGDRegressor()
    #model = linear_model.ElasticNet()
    #model = GradientBoostingRegressor()
    #RandomForestClassifier, RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
    model.fit(X_train_vec[train_ids], y_[train_ids])
    y_pred_valid = model.predict(X_train_vec[valid_ids])
    rmsle = get_rmsle(y_pred_valid, y_[valid_ids])
    print('valid rmsle: {}'.format(rmsle))
    models.append(model)
    
print("Make prediction")



test_predicts = np.ones(y_val.shape[0])
for m in models:
    test_predicts *= m.predict(X_val_vec)#.toarray())

test_predicts **= (1. / len(models))

print(get_rmsle(test_predicts, get_y(y_val)))

start...
valid rmsle: 4.917863703785783
valid rmsle: 4.706380091924154
valid rmsle: 5.858326230390174
valid rmsle: 4.7546670672005416
valid rmsle: 5.226447792403631
valid rmsle: 4.705981579785092
valid rmsle: 4.496186708472444
valid rmsle: 4.690858081837068
valid rmsle: 5.660884647191329
valid rmsle: 5.116432966705325
Make prediction


/home/algis/.local/lib/python3.5/site-packages/ipykernel/__main__.py:68: RuntimeWarning: invalid value encountered in power


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [17]:
def get_rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(np.expm1(y_true), np.expm1(y_pred)))

def get_ridge():
    return Ridge(
        solver='auto',
        fit_intercept=True,
        alpha=0.5,
        max_iter=100,
        normalize=False,
        tol=0.05)

cv = KFold(n_splits=10, shuffle=True, random_state=42)
for train_ids, valid_ids in cv.split(X_train_vec):
    model = RandomForestRegressor()
    model.fit(X_train_vec[train_ids], y_[train_ids])
    y_pred_valid = model.predict(X_train_vec[valid_ids])
    rmsle = get_rmsle(y_pred_valid, y_[valid_ids])
    print('valid rmsle: {}'.format(rmsle))
    break


valid rmsle: 6.352753139102475


In [ ]:
eli5.show_weights(model, vec=vectorizer)